In [1]:
#Detects enotion from text
#Created by S. Biswas
import pandas as pd

In [12]:
df_train = pd.read_csv('train.txt', sep = ';',   names = ['Sentence', 'Emotion'], encoding = 'UTF8')
print(df_train)
#data cleaning
df_train = df_train.dropna()
df_train = df_train.drop_duplicates
df_train = df_train[(df_train != "").all(axis=1)]

                                                Sentence  Emotion
0                                i didnt feel humiliated  sadness
1      i can go from feeling so hopeless to so damned...  sadness
2       im grabbing a minute to post i feel greedy wrong    anger
3      i am ever feeling nostalgic about the fireplac...     love
4                                   i am feeling grouchy    anger
...                                                  ...      ...
15995  i just had a very brief time in the beanbag an...  sadness
15996  i am now turning and i feel pathetic that i am...  sadness
15997                     i feel strong and good overall      joy
15998  i feel like this was such a rude comment and i...    anger
15999  i know a lot but i feel so stupid because i ca...  sadness

[16000 rows x 2 columns]


AttributeError: 'bool' object has no attribute 'all'

In [ ]:
df_train['length'] = len(sen) for sen in df_train